<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

Creating gs://cloud-training-demos-ml/...
ServiceException: 409 Bucket cloud-training-demos-ml already exists.


<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [16]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

## Lab Task #1

Sample the BigQuery resultset (above) so that you have approximately 12,000 training examples and 3000 evaluation examples.
The training and evaluation datasets have to be well-distributed (not all the babies are born in Jan 2005, for example)
and should not overlap (no baby is part of both training and evaluation datasets).

Hint (highlight to see): <p style='color:white'>You will use MOD() on the hashmonth to divide the dataset into non-overlapping training and evaluation datasets, and RAND() to sample these to the desired size.</p>

In [24]:
#check unique hashmonths and count for label
query1 = """
SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth
"""

In [26]:
df = bq.Query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").execute().result().to_dataframe()

In [28]:

print(" {} unique ids - HASMONTHS".format(df.shape[0]))
df.head()

 96 unique ids - HASMONTHS


,hashmonth,num_babies
0,2126480030009879160,344357
1,8904940584331855459,344191
2,6691862025345277042,338820
3,5934265245228309013,324598
4,1005902328334352553,337637


In [32]:
#creating defined train validation sets
df_train_query = "select * from (" + query + ") where MOD(hashmonth,4)<3 AND RAND() < 0.0005" 
df_eval_query = "select * from (" + query + ") where MOD(hashmonth,4)=3 AND RAND() < 0.0005" 
df_train = bq.Query(df_train_query).execute().result().to_dataframe()
df_eval = bq.Query(df_eval_query).execute().result().to_dataframe()
print(df_train.shape)
print(df_eval.shape)

(13302, 6)
(3377, 6)


In [34]:
print(df_train['hashmonth'].nunique())

77


## Lab Task #2

Use Pandas to:
* Clean up the data to remove rows that are missing any of the fields.
* Simulate the lack of ultrasound.
* Change the plurality column to be a string.

Hint (highlight to see): <p>
Filtering:
<pre style='color:white'>
df = df[df.weight_pounds > 0]
</pre>
Lack of ultrasound:
<pre style='color:white'>
nous = df.copy(deep=True)
nous['is_male'] = 'Unknown'
</pre>
Modify plurality to be a string:
<pre style='color:white'>
twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
df['plurality'].replace(twins_etc, inplace=True)
</pre>
</p>

In [35]:
df_train.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13288.000000,13302.000000,13302.000000,13230.000000,1.330200e+04
mean,7.229092,27.367163,1.036461,38.571051,4.414409e+18
std,1.327275,6.207734,0.197213,2.609007,2.805700e+18
min,0.500449,12.000000,1.000000,18.000000,1.244589e+17
25%,6.563162,22.000000,1.000000,38.000000,1.622638e+18
50%,7.312733,27.000000,1.000000,39.000000,4.329667e+18
75%,8.062305,32.000000,1.000000,40.000000,7.108882e+18
max,12.963181,49.000000,4.000000,47.000000,9.183606e+18


In [40]:
#creating preprocessing step for common
import pandas as pd
def preprocess(df):
  #filter
  df = df[df.weight_pounds >0 ]
  df = df[df.mother_age >0]
  df = df[df.plurality >0]
  df = df[df.gestation_weeks > 0]
  
  #plurality to categorical variable
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  
  #lack of ultra sound
  no_us = df.copy(deep = True)
  no_us.loc[no_us['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  no_us['is_male'] = 'Unknown'
  
  return pd.concat([df, no_us])
  
  
  

In [42]:

df_train_p = preprocess(df_train)
df_eval_p = preprocess(df_eval)
print(df_train_p.shape)
print(df_train.shape)

(26436, 6)
(13302, 6)


In [43]:
df_train_p.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,26436.000000,26436.000000,26436.000000,2.643600e+04
mean,7.228751,27.366167,38.575427,4.411415e+18
std,1.325565,6.209359,2.596112,2.803677e+18
min,0.500449,12.000000,18.000000,1.244589e+17
25%,6.563162,22.000000,38.000000,1.622638e+18
50%,7.312733,27.000000,39.000000,4.329667e+18
75%,8.062305,32.000000,40.000000,7.108882e+18
max,12.963181,49.000000,47.000000,9.183606e+18


## Lab Task #3

Write the cleaned out data into CSV files.  Change the name of the Pandas dataframes (traindf, evaldf) appropriately.



In [44]:
df_train_p.to_csv('train.csv', index=False, header=False)
df_eval_p.to_csv('eval.csv', index=False, header=False)

In [45]:
%bash
wc -l *.csv
head *.csv
tail *.csv

   6680 eval.csv
  26436 train.csv
  33116 total
==> eval.csv <==
7.7272022831,False,19,Single(1),40.0,2246942437170405963
8.4326815215,True,39,Single(1),41.0,7146494315947640619
7.62578964258,False,32,Single(1),39.0,6244544205302024223
7.22675294836,False,35,Single(1),40.0,8904940584331855459
8.1019881285,True,26,Single(1),39.0,1569531340167098963
5.84665918824,True,20,Single(1),39.0,6365946696709051755
8.18796841068,True,29,Single(1),40.0,1088037545023002395
7.1870697412,False,22,Single(1),41.0,1569531340167098963
7.14738653404,False,33,Single(1),39.0,2246942437170405963
5.8135898489399995,True,32,Single(1),37.0,6910174677251748583

==> train.csv <==
6.87401332916,True,25,Single(1),40.0,774501970389208065
6.7130758779,True,28,Single(1),37.0,774501970389208065
9.37626000286,False,23,Single(1),41.0,774501970389208065
7.4406013425,True,19,Single(1),37.0,774501970389208065
6.686620406459999,False,25,Single(1),39.0,774501970389208065
8.1240343547,False,22,Single(1),40.0,774501970389208065

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License